# <font color='red'>Data Analysis Project</font>
## <font color='red'>E-commerce dataset from Kaggle.com</font>

Packages loading

In [350]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Data loading

In [351]:
df_1 = pd.read_csv('data.csv', encoding='ISO-8859-1')
df_1.shape

(541909, 8)

In [352]:
df_1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [353]:
# Adicionar a coluna TotalPrice ao dataframe df_1
df_1['TotalPrice'] = df_1['Quantity'] * df_1['UnitPrice']

# Exibir as primeiras linhas para verificar se a coluna foi adicionada corretamente
print(df_1.head())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom       15.30  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom       20.34  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom       22.00  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom       20.34  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom       20.34  


In [354]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
 8   TotalPrice   541909 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 37.2+ MB


In [355]:
df_1.describe()

,Quantity,UnitPrice,CustomerID,TotalPrice
count,541909.000000,541909.000000,406829.000000,541909.000000
mean,9.552250,4.611114,15287.690570,17.987795
std,218.081158,96.759853,1713.600303,378.810824
min,-80995.000000,-11062.060000,12346.000000,-168469.600000
25%,1.000000,1.250000,13953.000000,3.400000
50%,3.000000,2.080000,15152.000000,9.750000
75%,10.000000,4.130000,16791.000000,17.400000
max,80995.000000,38970.000000,18287.000000,168469.600000


In [356]:
# Cálculo do IQR para Quantity
Q1_quantity = df_1['Quantity'].quantile(0.25)
Q3_quantity = df_1['Quantity'].quantile(0.75)
IQR_quantity = Q3_quantity - Q1_quantity

# Definir limites para Quantity
lower_bound_quantity = Q1_quantity - 1.5 * IQR_quantity
upper_bound_quantity = Q3_quantity + 1.5 * IQR_quantity
print(lower_bound_quantity, upper_bound_quantity)

# Cálculo do IQR para UnitPrice
Q1_unitprice = df_1['UnitPrice'].quantile(0.25)
Q3_unitprice = df_1['UnitPrice'].quantile(0.75)
IQR_unitprice = Q3_unitprice - Q1_unitprice

# Definir limites para UnitPrice
lower_bound_unitprice = Q1_unitprice - 1.5 * IQR_unitprice
upper_bound_unitprice = Q3_unitprice + 1.5 * IQR_unitprice
print(lower_bound_unitprice, upper_bound_unitprice)

-12.5 23.5
-3.0700000000000003 8.45


In [357]:
# Filtrar outliers em Quantity
df_1 = df_1[(df_1['Quantity'] >= lower_bound_quantity) & (df_1['Quantity'] <= upper_bound_quantity)]

# Filtrar outliers em UnitPrice
df_1 = df_1[(df_1['UnitPrice'] >= lower_bound_unitprice) & (df_1['UnitPrice'] <= upper_bound_unitprice)]

# Exibir a quantidade de registros após a remoção dos outliers
print(f"Quantidade de registros após a remoção dos outliers: {df_1.shape[0]}")

Quantidade de registros após a remoção dos outliers: 444108


In [358]:
df_1[['Quantity', 'UnitPrice', 'TotalPrice']].describe()

,Quantity,UnitPrice,TotalPrice
count,444108.000000,444108.000000,444108.000000
mean,4.803066,2.694544,10.470162
std,4.616762,1.930320,11.705544
min,-12.000000,0.000000,-97.800000
25%,1.000000,1.250000,2.950000
50%,3.000000,2.080000,6.800000
75%,8.000000,3.750000,15.000000
max,23.000000,8.330000,191.590000


Treating rows with 'Quantity' or 'UnitPrice' less or equal to zero

In [359]:
# Contar registros com Quantity ou UnitPrice iguais ou menores do que zero
columns_zero = ((df_1['Quantity'] <= 0) | (df_1['UnitPrice'] <= 0)).sum()

# Exibir os resultados
print(f"Quantidade de registros com Quantity ou UnitPrice iguais ou menores do que zero: {columns_zero}")

Quantidade de registros com Quantity ou UnitPrice iguais ou menores do que zero: 7942


In [360]:
# Exibir a quantidade de registros antes da remoção
print(f"Quantidade de registros antes da remoção: {df_1.shape[0]}")

# Deletar os registros onde UnitPrice é menor ou igual a zero
df_1 = df_1[df_1['UnitPrice'] > 0]

# Deletar os registros onde Quantity é menor ou igual a zero
df_1 = df_1[df_1['Quantity'] > 0]

# Exibir a quantidade de registros restantes para verificação
print(f"Quantidade de registros após a remoção: {df_1.shape[0]}")

Quantidade de registros antes da remoção: 444108
Quantidade de registros após a remoção: 436166


In [361]:
df_1[['Quantity', 'UnitPrice', 'TotalPrice']].describe()

,Quantity,UnitPrice,TotalPrice
count,436166.000000,436166.000000,436166.000000
mean,4.938260,2.699178,10.785505
std,4.508586,1.927294,11.514780
min,1.000000,0.001000,0.001000
25%,1.000000,1.250000,3.290000
50%,3.000000,2.080000,7.460000
75%,8.000000,3.750000,15.300000
max,23.000000,8.330000,191.590000


Treating rows with 'TotalPrice' less than 0.01

In [362]:
# Contar registros com TotalPrice menor do que 0.01
quant_sm = (df_1['TotalPrice'] < 0.01).sum()

# Exibir os resultados
print(f"Quantidade de registros com TotalPrice menor do que 0.01 é: {quant_sm}")

# Filtrar registros com TotalPrice menor do que 0.01
df_1 = df_1[(df_1['TotalPrice'] >= 0.01)]

Quantidade de registros com TotalPrice menor do que 0.01 é: 4


Treating rows with 'Quantity' greater than 4800

In [363]:
df_1[['Quantity', 'UnitPrice', 'TotalPrice']].describe()

,Quantity,UnitPrice,TotalPrice
count,436162.000000,436162.000000,436162.000000
mean,4.938296,2.699202,10.785604
std,4.508591,1.927285,11.514787
min,1.000000,0.060000,0.060000
25%,1.000000,1.250000,3.290000
50%,3.000000,2.080000,7.460000
75%,8.000000,3.750000,15.300000
max,23.000000,8.330000,191.590000


In [364]:
duplicados = df_1[df_1.duplicated(keep=False)]
print(duplicados)

       InvoiceNo StockCode                       Description  Quantity  \
485       536409     22111      SCOTTIE DOG HOT WATER BOTTLE         1   
489       536409     22866     HAND WARMER SCOTTY DOG DESIGN         1   
494       536409     21866       UNION JACK FLAG LUGGAGE TAG         1   
517       536409     21866       UNION JACK FLAG LUGGAGE TAG         1   
521       536409     22900   SET 2 TEA TOWELS I LOVE LONDON          1   
...          ...       ...                               ...       ...   
541675    581538     22068       BLACK PIRATE TREASURE CHEST         1   
541689    581538     23318    BOX OF 6 MINI VINTAGE CRACKERS         1   
541692    581538     22992            REVOLVER WOODEN RULER          1   
541699    581538     22694                      WICKER STAR          1   
541701    581538     23343      JUMBO BAG VINTAGE CHRISTMAS          1   

            InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
485     12/1/2010 11:45       4.95

In [365]:
df_1.drop_duplicates(keep='last', inplace=True) 

In [366]:
duplicados = df_1[df_1.duplicated(keep='first')]
print(duplicados)

Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, TotalPrice]
Index: []


In [367]:
df_1[['Quantity', 'UnitPrice', 'TotalPrice']].describe()

,Quantity,UnitPrice,TotalPrice
count,431218.000000,431218.000000,431218.000000
mean,4.970481,2.705439,10.864298
std,4.515114,1.929730,11.542093
min,1.000000,0.060000,0.060000
25%,1.000000,1.250000,3.290000
50%,3.000000,2.080000,7.500000
75%,8.000000,3.750000,15.300000
max,23.000000,8.330000,191.590000


In [368]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 431218 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    431218 non-null  object 
 1   StockCode    431218 non-null  object 
 2   Description  431218 non-null  object 
 3   Quantity     431218 non-null  int64  
 4   InvoiceDate  431218 non-null  object 
 5   UnitPrice    431218 non-null  float64
 6   CustomerID   316089 non-null  float64
 7   Country      431218 non-null  object 
 8   TotalPrice   431218 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 32.9+ MB


In [381]:
df_1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [388]:
count_customers = (df_1['CustomerID'].isna()).sum()
print(count_customers)

115129


In [389]:
# Filtrar as linhas onde CustomerID está faltante (NaN)
missing_customerid_df = df_1[df_1['CustomerID'].isna()]

# Exibir as primeiras linhas para verificar
print(missing_customerid_df.head())

# Contar quantos registros têm CustomerID faltante
missing_count = missing_customerid_df.shape[0]
print(f"Quantidade de registros com CustomerID faltante: {missing_count}")


     InvoiceNo StockCode                      Description  Quantity  \
1443    536544     21773  DECORATIVE ROSE BATHROOM BOTTLE         1   
1444    536544     21774  DECORATIVE CATS BATHROOM BOTTLE         2   
1445    536544     21786               POLKADOT RAIN HAT          4   
1446    536544     21787            RAIN PONCHO RETROSPOT         2   
1447    536544     21790               VINTAGE SNAP CARDS         9   

          InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
1443  12/1/2010 14:32       2.51         NaN  United Kingdom        2.51  
1444  12/1/2010 14:32       2.51         NaN  United Kingdom        5.02  
1445  12/1/2010 14:32       0.85         NaN  United Kingdom        3.40  
1446  12/1/2010 14:32       1.66         NaN  United Kingdom        3.32  
1447  12/1/2010 14:32       1.66         NaN  United Kingdom       14.94  
Quantidade de registros com CustomerID faltante: 115129
